In [ ]:
# lib/fast_rcnn/nums_wrapper.py
from fast_rcnn.config import cfg
from nms.gpu_nms import gpu_nms
from nms.cpu_nms import cpu_nms

def nms(dets, thresh):
    """Dispatch to either CPU or GPU NMS implementations."""

    if dets.shape[0] == 0:
        return []
    if cfg.USE_GPU_NMS:
        return gpu_nms(dets, thresh, device_id=cfg.GPU_ID)
    else:
        return cpu_nms(dets, thresh)

In [37]:
#lib/nms/py_cpu_nms.py
def py_cpu_nms(dets, thresh):
    """Pure Python NMS baseline."""
    # all detections(dets) are collected into:
    #    all_boxes[cls][image] = N x 5 array of detections in
    #    (x1, y1, x2, y2, score)
    #TopLeft:(x1、y1)
    #ButtomRight:(x2、y2) and the score
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    # the dets' areas
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    #order: sort by score (large to small) and get the indexes
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]  # the index in bets (or in the remain order) with the largest score
        keep.append(i) # keep this index
        
        #the top left point of intersection area
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        #the buttom right point of intersection area
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        # calculate the area of intersection area
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        #calculate：the area of intersection/（area with the largest score+ area-intersection）
        # len(ovr) = len(order[1:])
        # index of ovr = index of order[1:] = index of order + 1
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= thresh)[0]
        
        order = order[inds + 1]
    
    return keep

In [54]:
dets = np.array([[0,0,2,2,10],
                [1,1,3,3,20],
                [0,0,2,2,100],
                [1,1,3,3,200]])

In [55]:
py_cpu_nms(dets,0.5)

[3, 2]

In [56]:
py_cpu_nms(dets,0.1)

[3]

In [57]:
x1 = dets[:, 0]
y1 = dets[:, 1]
x2 = dets[:, 2]
y2 = dets[:, 3]
scores = dets[:, 4]

areas = (x2 - x1 + 1) * (y2 - y1 + 1)
order = scores.argsort()[::-1]

In [58]:
x1

array([0, 1, 0, 1])

In [59]:
y1

array([0, 1, 0, 1])

In [60]:
x2

array([2, 3, 2, 3])

In [61]:
y2

array([2, 3, 2, 3])

In [62]:
scores

array([ 10,  20, 100, 200])

In [63]:
areas  #

array([9, 9, 9, 9])

In [64]:
dets

array([[  0,   0,   2,   2,  10],
       [  1,   1,   3,   3,  20],
       [  0,   0,   2,   2, 100],
       [  1,   1,   3,   3, 200]])

In [65]:
dets[::-1]

array([[  1,   1,   3,   3, 200],
       [  0,   0,   2,   2, 100],
       [  1,   1,   3,   3,  20],
       [  0,   0,   2,   2,  10]])

In [66]:
scores.argsort()  # small to large

array([0, 1, 2, 3])

In [67]:
scores.argsort()[::-1]  # large to small

array([3, 2, 1, 0])

In [68]:
order = scores.argsort()[::-1]

In [71]:
    thresh = 0.5    
    keep = []
    while order.size > 0:
        i = order[0]  # the index in bets (or in the remain order) with the largest score
        keep.append(i) # keep this index
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        #计算相交的面积
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        #计算：重叠面积/（面积1+面积2-重叠面积）
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

In [73]:
keep

[3, 2]

In [79]:
        order = scores.argsort()[::-1]
        print(order)
        thresh = 0.5    
        keep = []
        i = order[0]  # the index in bets (or in the remain order) with the largest score
        keep.append(i) # keep this index
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        #计算相交的面积
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        #计算：重叠面积/（面积1+面积2-重叠面积）
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        print(ovr)

        inds = np.where(ovr <= thresh)[0]
        print(inds)
        order = order[inds + 1]
        print(order)

[3 2 1 0]
[ 0.28571429  1.          0.28571429]
[0 2]
[2 0]


In [76]:
order

array([2, 0])